In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

class MovielensDataset(Dataset):
    def __init__(self, path, transform=None):
        self.ratings = pd.read_csv(path).values
        self.path = path
        self.transform= transform
        
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        """
        Returns uid, iid and rating
        """
        return int(self.ratings[idx][0]), int(self.ratings[idx][1]), float(self.ratings[idx][2])

In [ ]:
train_dataset = MovielensDataset(path="../data/df_train.csv", transform=ToTensor())
validation_dataset = MovielensDataset(path="../data/df_val.csv", transform=ToTensor())
test_dataset = MovielensDataset(path="../data/df_test.csv", transform=ToTensor())

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=256, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Matrix Factorization with Visual Information

In [ ]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=30):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.user_bias = torch.nn.Embedding(n_users, 1)
            
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.item_bias = torch.nn.Embedding(n_items, 1) 

        for param in self.parameters():
            torch.nn.init.normal_(param, std=0.01)  
    
    def forward(self, user, item):
        Q = self.user_factors(user)
        bq = self.user_bias(user).flatten()

        I = self.item_factors(item)
        bi = self.item_bias(item).flatten()

        return (Q*I).sum(-1) + bq + bi